In [ ]:
%nbdev_default_export control.functions
%load_ext autoreload
%autoreload 2

Cells will be exported to pct.control.functions,
unless a different module is specified after an export flag: `%nbdev_export special.module`


In [ ]:
from abc import ABC, abstractmethod

In [ ]:
%nbdev_export
class BaseFunction(ABC):
    "Base class of a PCT function."
    def __init__(self):
        self.output = 0
    
    @abstractmethod
    def get_output(self):
        pass

In [ ]:
class Integrator(BaseFunction):
    "Integrator function."
    def __init__(self, gain, slow):
        self.gain = gain
        self.slow = slow
 

In [ ]:
i = Integrator(1, 10)

TypeError: Can't instantiate abstract class Integrator with abstract methods get_output

In [ ]:
from nbdev import *
notebook2script()

Converted 00_core.ipynb.
Converted 01_rmath.ipynb.
Converted 02_functions.ipynb.
Converted index.ipynb.
